In [18]:
import os
import pandas as pd
SCORE_DIR = "data/scores"

In [2]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [3]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import pandas as pd

SCORE_DIR = "data/scores"
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

games_df = pd.concat(games, ignore_index=True)
games_df

games_df.to_csv("nba_games.csv")

100 / 9973
200 / 9973
300 / 9973
400 / 9973
500 / 9973
600 / 9973
700 / 9973
800 / 9973
900 / 9973
1000 / 9973
1100 / 9973
1200 / 9973
1300 / 9973
1400 / 9973
1500 / 9973
1600 / 9973
1700 / 9973
1800 / 9973
1900 / 9973
2000 / 9973
2100 / 9973
2200 / 9973
2300 / 9973
2400 / 9973
2500 / 9973
2600 / 9973
2700 / 9973
2800 / 9973
2900 / 9973
3000 / 9973
3100 / 9973
3200 / 9973
3300 / 9973
3400 / 9973
3500 / 9973
3600 / 9973
3700 / 9973
3800 / 9973
3900 / 9973
4000 / 9973
4100 / 9973
4200 / 9973
4300 / 9973
4400 / 9973
4500 / 9973
4600 / 9973
4700 / 9973
4800 / 9973
4900 / 9973
5000 / 9973
5100 / 9973
5200 / 9973
5300 / 9973
5400 / 9973
5500 / 9973
5600 / 9973
5700 / 9973
5800 / 9973
5900 / 9973
6000 / 9973
6100 / 9973
6200 / 9973
6300 / 9973
6400 / 9973
6500 / 9973
6600 / 9973
6700 / 9973
6800 / 9973
6900 / 9973
7000 / 9973
7100 / 9973
7200 / 9973
7300 / 9973
7400 / 9973
7500 / 9973
7600 / 9973
7700 / 9973
7800 / 9973
7900 / 9973
8000 / 9973
8100 / 9973
8200 / 9973
8300 / 9973
8400 / 9973
8